In [1]:
%load_ext autoreload
%autoreload 2


import os
os.chdir('/Users/samet/projects/imbalanced-data/fraud_detection')
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split, learning_curve
from sklearn.metrics import average_precision_score
from sklearn.pipeline import Pipeline, FeatureUnion
from xgboost.sklearn import XGBClassifier
import src.transformers as trans
import yaml


In [2]:
with open('./ml_config.yaml', "r") as f:
    config = yaml.load(f, Loader=yaml.FullLoader)

In [3]:
df_train = pd.read_csv(config["TRAIN_DATA"], nrows=5e5)
y_train = df_train.pop(config["TARGET"])
X_train = df_train


df_test = pd.read_csv(config["TEST_DATA"])
y_test = df_test.pop(config["TARGET"])
X_test = df_test


In [4]:
pipe = Pipeline([
    ('drop_columns', trans.KeepFeatures(config["FEATURES"])),
    ('label_encode', trans.CustomLabelEncoder(config["LABEL_ENCODE_FEATURES"])),
    ('amount_vs_old_new_balance_dest', trans.AmountVsOldAndNewBalanceDest()),
    ('amount_vs_old_new_balance_orig', trans.AmountVsOldAndNewBalanceOrig()),
    ('xgbclassifier', XGBClassifier(max_depth = 1, scale_pos_weight = 99, n_jobs = -1)) ,
])


pipe.fit(X_train, y_train)

# grid = GridSearchCV(estimator=pipe, param_grid=param_grid, n_jobs=-1, cv=cv, scoring='roc_auc')
#https://discuss.xgboost.ai/t/how-does-scale-pos-weight-affect-probabilities/1790

Pipeline(steps=[('drop_columns',
                 KeepFeatures(features=['step', 'type', 'amount',
                                        'oldBalanceOrig', 'newBalanceOrig',
                                        'oldBalanceDest', 'newBalanceDest'])),
                ('label_encode', CustomLabelEncoder(features=['type'])),
                ('amount_vs_old_new_balance_dest',
                 AmountVsOldAndNewBalanceDest()),
                ('amount_vs_old_new_balance_orig',
                 AmountVsOldAndNewBalanceOrig()),
                ('xgbclassifie...
                               feature_types=None, gamma=0, gpu_id=-1,
                               grow_policy='depthwise', importance_type=None,
                               interaction_constraints='',
                               learning_rate=0.300000012, max_bin=256,
                               max_cat_threshold=64, max_cat_to_onehot=4,
                               max_delta_step=0, max_depth=1, max_leaves=0,
                               min_child_weight=1, missing=nan,
                               monotone_constraints='()', n_estimators=100,
                               n_jobs=-1, num_parallel_tree=1, predictor='auto',
                               random_state=0, ...))])

In [5]:
probabilities = pipe.predict_proba(X_test)
print('AUPRC = {}'.format(average_precision_score(y_test, probabilities[:, 1]>.5))) 


AUPRC = 0.99657431296194


In [9]:
from pydantic import BaseModel

In [24]:
from typing import List

class InputData(BaseModel):
    step: int
    type: object
    amount: float
    oldBalanceOrig: float
    newBalanceOrig: float
    oldBalanceDest: float
    newBalanceDest: float


class OutputData(BaseModel):
    output:bool






temp = pipe[0].transform(X_test)
X_temp = temp.query('step == 500')
y_temp = y_test.loc[X_temp.index]
X_temp
y_temp.sum()

data = X_temp.iloc[0].to_dict()


InputData(**data)

InputData(step=500, type='CASH_OUT', amount=163689.47, oldBalanceOrig=32453.0, newBalanceOrig=0.0, oldBalanceDest=478209.94, newBalanceDest=641899.41)

In [63]:
data = {'step': 500,
 'type': 'CASH_OUT',
 'amount': 163689.47,
 'oldBalanceOrig': 32453.0,
 'newBalanceOrig': 0.0,
 'oldBalanceDest': 478209.94,
 'newBalanceDest': 641899.41}
 
new_df = pd.DataFrame(InputData(**data).dict(), index=[0])
display(new_df)

is_fraud = pipe.predict_proba(new_df)[0][1]>.5
is_fraud

,step,type,amount,oldBalanceOrig,newBalanceOrig,oldBalanceDest,newBalanceDest
0,500,CASH_OUT,163689.47,32453.0,0.0,478209.94,641899.41


False

In [71]:
data = X_temp.iloc[:2].values
data

array([[500, 'CASH_OUT', 163689.47, 32453.0, 0.0, 478209.94, 641899.41],
       [500, 'CASH_OUT', 1106.62, 100629.0, 99522.38, 110979.12,
        112085.74]], dtype=object)

In [ ]:

# drop functioni degistir, keep yap

In [ ]:
def score(data:InputData):
    model_input = np.array([v for k,v in data.dict().items()]).reshape(1,-1)
    result = model.predict_proba(model_input)[:,-1]
    
    return {'score':result}